In [5]:
import pika;
import json;
import time;

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re

# selenium=4.24.0

In [7]:
MQ_server = '192.168.0.202'
driver_path = 'C:\\chromedriver-win64\\chromedriver.exe'
extension_path = 'C:\\chromedriver-win64\\bypass-paywalls-firefox-clean-master.crx'

In [10]:
service = Service(executable_path=driver_path)
option = Options()
option.add_extension(extension=extension_path)
web = webdriver.Chrome(service=service,options=option)

In [11]:
def webGet(url, title):
    web.get(url=url)
    textdata = web.find_elements(By.CLASS_NAME, 'evys1bk0')
    artifact = ''
    for data in textdata:
        artifact = artifact + data.text
    safe_filename = re.sub(r'[<>:"/\\|?*]', '_', title) + ".txt"
    with open(safe_filename,"w") as file:
        file.write(artifact)


In [13]:
connection = pika.BlockingConnection(pika.ConnectionParameters(MQ_server))
channel = connection.channel()
channel.basic_qos(prefetch_count=1)
try:
    while True:
        method_frame, header_frame, body = channel.basic_get(queue='nytime', auto_ack=False)

        if method_frame:
            data = json.loads(body)
            print(f" [x] Received title={data['title']}.\n author={data['author']},\n time = {data['date']}")
            webGet(data['url'], data['title'])
            time.sleep(10)
            channel.basic_ack(delivery_tag=method_frame.delivery_tag)
        else:
            print(" [*] No messages in queue.")
            time.sleep(10)
except KeyboardInterrupt:
    print("\n [*] Exiting.")
finally:
    connection.close()

 [x] Received title=Trump’s U.S.A.I.D. Cuts Hobble Earthquake Response in Myanmar.
 author=Hannah Beech and Edward Wong,
 time = Sun, 30 Mar 2025 20:49:35 +0000
 [x] Received title=Gananath Obeyesekere, 95, Dies; Anthropologist Bridged East and West.
 author=Clay Risen,
 time = Sun, 30 Mar 2025 15:33:28 +0000
 [x] Received title=Hegseth Pledges to Step Up Military Cooperation With Japan and Deter China.
 author=Martin Fackler,
 time = Sun, 30 Mar 2025 15:41:41 +0000
 [x] Received title=In Myanmar, Earthquake Aftershocks and Airstrikes Terrorize Residents.
 author=Sui-Lee Wee,
 time = Sun, 30 Mar 2025 14:39:24 +0000
 [x] Received title=Bangkok Has New Fears About High-Rise Buildings After Myanmar Earthquake.
 author=Damien Cave, Muktita Suhartono and Richard C. Paddock,
 time = Sun, 30 Mar 2025 16:56:16 +0000
 [x] Received title=In Tokyo, Rice Farmers Protest ‘Misguided’ Rules Fueling Shortages.
 author=River Akira Davis and Hisako Ueno,
 time = Sun, 30 Mar 2025 09:19:27 +0000
 [x] Rece